All the libraries that we will be using for these bits of code have been mentioned. 

The process is as follows : 
    1. Develop a function to get the information we need from a webpage 
    2. Develop a function to clean up the information we get from the webpage 
    3. Get a set of URLs we want to get information from into a list  
    4. Run the above two functions for each URL in our list 
    5. Save the results to a csv file 

In [2]:
#Indeed web scraper for job listings to apply for 
#import urllib
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re
import datetime
from datetime import timedelta
from tqdm import tqdm

In [208]:
# parse the title , location , company , salary , synopsis , date of posting and link 
def parse(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
    df = pd.DataFrame(columns=["Title","Location","Company","Salary", "Synopsis","date","href"])

    for each in soup.find_all(class_= "result" ):
            # get all the required data fields 
            try: 
                title = each.find(class_='jobtitle').text.replace('\n', '')
            except:
                title = 'None'
            try: 
                href = each.find(class_='jobtitle').find('a')['href']
            except:
                href = 'None'

            try:
                location = each.find('span', {'class':"location" }).text.replace('\n', '')
            except:
                location = 'None'
            try: 
                company = each.find(class_='company').text.replace('\n', '')
            except:
                company = 'None'
            try:
                salary = each.find('span', {'class':'no-wrap'}).text.replace('\n', '')
            except:
                salary = 'None'
            try: 
                date = each.find('span', {'class':'date'}).text.replace('\n', '')
            except:
                date = 'None'
            synopsis = each.find('span', {'class':'summary'}).text.replace('\n', '')
            df = df.append({'href':href,'date':date, 'Title':title, 'Location':location, 'Company':company, 'Salary':salary, 'Synopsis':synopsis}, ignore_index=True)
    return df

In [222]:
# clean up the dataframe , change data columns to what I need 

def transform_columns(link):
    y = parse(link)
    # replace the href to go to the job posting link 
    y['href'] = y['href'].replace('rc/clk','viewjob')
    y['href'] = 'http://www.indeed.com/' + y['href']
    
    
    # change the date to reflect the timestamp 
    y['date'] = y['date'].str.replace(' day ago','')
    y['date'] = y['date'].str.replace(' days ago','')
    y['date'] = y['date'].str.replace('Just posted','0')
    y['date'] = y['date'].str.replace('Today','0')
    y['date'] = y['date'].str.replace('None','30')
    y['date'] = y['date'].str.replace('+','')
    y.loc[y['date'].str.contains('hour', case=False), 'date'] = '0'
    y.loc[y['date'].str.contains('minutes', case=False), 'date'] = '0'
    y['date'] = y['date'].astype(str).astype(int)
    y['date'] = datetime.datetime.now().date() -  pd.to_timedelta(y['date'], unit='d')

    return y

In [223]:
url_template = "http://www.indeed.com/jobs?q={}+%2420%2C000&l={}&start={}"
links = []

# define all the roles , locations , number of results for which I want to apply 

roles = ['data+scientist','data+engineer','machine+learning','data+analyst']

locations = ['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle', 
    'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 
    'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', 'Washington+DC', 
    'Charlottesville', 'Richmond', 'Baltimore', 'Harrisonburg', 'San+Antonio', 'San+Diego', 'San+Jose'
    'Austin', 'Jacksonville', 'Indianapolis', 'Columbus', 'Fort+Worth', 'Charlotte', 'Detroit', 'El+Paso', 
    'Memphis', 'Boston', 'Nashville', 'Louisville', 'Milwaukee', 'Las+Vegas', 'Albuquerque', 'Tucson', 
    'Fresno', 'Sacramento', 'Long+Beach', 'Mesa', 'Virginia+Beach', 'Norfolk', 'Atlanta', 'Colorado+Springs',
    'Raleigh', 'Omaha', 'Oakland', 'Tulsa', 'Minneapolis', 'Cleveland', 'Wichita', 'Arlington', 'New+Orleans', 
    'Bakersfield', 'Tampa', 'Honolulu', 'Anaheim', 'Aurora', 'Santa+Ana', 'Riverside', 'Corpus+Christi', 'Pittsburgh', 
    'Lexington', 'Anchorage', 'Cincinnati', 'Baton+Rouge', 'Chesapeake', 'Alexandria', 'Fairfax', 'Herndon',
    'Reston', 'Roanoke','Orlando']
start = ['0','50','100','150','200']

for i in range(0,len(roles)):
    for j in range(0,len(locations)):
        for k in range(0,len(start)):
            links.append(url_template.format(roles[i],locations[j],start[k]))







In [224]:

df = pd.DataFrame(columns=["Title","Location","Company","Salary", "Synopsis","date","href"])

for i in tqdm(range(0,len(links))):
    df = df.append(transform_columns(links[i]))
    # drop duplicate entries based on synopsis and company name 
    # the logic here is that these two are enough to identify what the unique entries are . 
    
    df = df.drop_duplicates(["Company","Synopsis"])
    time.sleep(2)
print('finished parsing')




100%|██████████| 1054/1054 [48:12<00:00,  2.74s/it]


finished parsing


In [225]:
df.to_csv('test1.csv',index=False,encoding='utf-8')

In [ ]:
#The jane austen word counter logic  
# download the book . Project Gutenberg has a txt version of the entire book . Get it . 
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [11]:
book = requests.get('https://www.gutenberg.org/files/1342/1342.txt').content

In [13]:
book = str(book)

In [27]:
def format_sentence(sent):
    return({word: True for word in nltk.word_tokenize(sent)})

In [30]:
# format_sentence("this is a sentence")

'nltk.download' is not recognized as an internal or external command,
operable program or batch file.


In [34]:
from bs4 import BeautifulSoup
from collections import Counter
import requests
from string import whitespace, punctuation
r = requests.get("https://www.gutenberg.org/files/1342/1342-h/1342-h.htm")

soup = BeautifulSoup(r.content, "html.parser")

text = (word.lower().strip(whitespace + punctuation)
        for element in soup.findAll('p')
        for text in element.findAll(text=True)
        for word in text.split())

words_to_exclude = list(set(stopwords.words('english')))
words_to_exclude.append('mr')
words_to_exclude.append('could')
words_to_exclude.append('would')
words_to_exclude.append('said')
words_to_exclude.append('mrs')
words_to_exclude.append('much')
words_to_exclude.append('must')
words_to_exclude.append('“i')
words_to_exclude.append('miss')
words_to_exclude.append('one')
words_to_exclude.append('know')


#{'knew','a', 'or', 'the', 'of', 'it', 'is', 'it', 'and', 'in', 'she', 'he', 'to', 'i',"i","me","my","myself","we","our","ours","ourselves","you","your","yours","yourself","yourselves","he","him","his","himself","she","her","hers","herself","it","its","itself","they","them","their","theirs","themselves","what","which","who","whom","this","that","these","those","am","is","are","was","were","be","been","being","have","has","had","having","do","does","did","doing","a","an","the","and","but","if","or","because","as","until","while","of","at","by","for","with","about","against","between","into","through","during","before","after","above","below","to","from","up","down","in","out","on","off","over","under","again","further","then","once","here","there","when","where","why","how","all","any","both","each","few","more","most","other","some","such","no","nor","not","only","own","same","so","than","too","very","s","t","can","will","just","don","should","now",'ma\'am'}

important = Counter(word for word in text if word not in words_to_exclude)

print(important.most_common(6))



[('elizabeth', 587), ('darcy', 349), ('bennet', 279), ('jane', 252), ('bingley', 243), ('never', 219)]


In [23]:
from nltk.corpus import stopwords
import nltk
# ( if you don't have the corpus already downloaded , run this once')
# nltk.download('stopwords')
list1 = list(set(stopwords.words('english')))

'herself'